# Knowledge Structure Graph Generator

## 1. DB연결

In [16]:
import pymysql

In [17]:
juso_db = pymysql.connect(
    user='root', 
    passwd='1234', 
    host='127.0.0.1', 
    db='test', 
    charset='utf8'
)

In [18]:
cursor = juso_db.cursor(pymysql.cursors.DictCursor)

In [19]:
sql = "SELECT * FROM knowledge_structure;"
cursor.execute(sql)
result = cursor.fetchall()

In [20]:
import pandas as pd

result = pd.DataFrame(result)
print("------------------------------지식구조 목록 리스트------------------------------")
print(result) #지식구조 목록 리스트화

------------------------------지식구조 목록 리스트------------------------------
  DOC_ID COT_MODE                                KNOWLEDGE_STRUCTURE
0     감초      scs  부작용,기저,1.8\n약용,식품,3.21\n하루,이상,3.9\n이상,단맛,3.9\n...
1     대마      scs  마약,법률,3.06\n약물,중독,3.26\n합법,국가,3.38\n필로폰,헤로인,4....
2      차      scs  남부,지방,2.71\n지역,재배,4.03\n우려,식물,4.81\n지역,남부,4.85...
3      콩      scs  두부,된장,1.8\n고기,짜파게티,2.53\n여성,에스트로겐,3.0\n단백질,부족,...
4     홍차      scs  우려,시간,3.79\n커피,여성,3.79\n홍차,영국,4.68\n시간,한국,4.69...
5     황기      scs  독성,약재,1.0\n이상,한약재,1.0\n이상,의약품,1.0\n한약재,의약품,1.0...


In [21]:
knowledge_structure = result['KNOWLEDGE_STRUCTURE']

## 2. 데이터 선택

#### 작성할 파일 제목 정하기
* file_name = "파일명"

In [22]:
file_name = "홍차" 

#### 추출하기 원하는 인덱스 선정

In [23]:
data = knowledge_structure[4] #인덱스만 변경하기
print(data)

우려,시간,3.79
커피,여성,3.79
홍차,영국,4.68
시간,한국,4.69
영국,인도,4.78
티백,경우,4.92
영국,중국,5.02
인도,소비,5.04
영국,청나라,5.08
국내,한국,5.16
녹차,국내,5.23
경우,카페,5.4
홍차,귀족,5.47
판매,카페,5.49
홍차,국내,5.56
홍차,카페,5.68
커피,경우,5.78
티백,설탕,6.35
티백,사용,6.56



In [24]:
import re

contents = re.split('\n', data)
contents = contents[:-1] #맨 끝 데이터는 공백

In [25]:
#노드 추가
for content in contents:
    value = re.split(',', content) # ,단위로 구분하여 리스트화
    print(value)

['우려', '시간', '3.79']
['커피', '여성', '3.79']
['홍차', '영국', '4.68']
['시간', '한국', '4.69']
['영국', '인도', '4.78']
['티백', '경우', '4.92']
['영국', '중국', '5.02']
['인도', '소비', '5.04']
['영국', '청나라', '5.08']
['국내', '한국', '5.16']
['녹차', '국내', '5.23']
['경우', '카페', '5.4']
['홍차', '귀족', '5.47']
['판매', '카페', '5.49']
['홍차', '국내', '5.56']
['홍차', '카페', '5.68']
['커피', '경우', '5.78']
['티백', '설탕', '6.35']
['티백', '사용', '6.56']


## 3. Data JavaScript File 생성

### 파일 2개 생성
* keyword.js : data(nodes, edges)
* keyword.html : graph

### 01. keyword.js

In [26]:
import networkx as nx

In [27]:
G = nx.Graph()

In [28]:
# 에지리스트를 그래프에 넣기 : 그래프에 입력된 노드 확인용
for content in contents:
    tmp = re.split(',', content)
    G.add_edge(tmp[0], tmp[1], weight = tmp[2])

In [29]:
degree_list = [degree for degree in G.degree()] # 각 노드별 차수 저장

In [30]:
# 파일 저장용 nodes, edges
graph_nodes = "var nodes = "
graph_edges = "var edges = "

# 리스트 생성용
num = 0
nodes_list = []
reverse_graph = dict()

for name in G.nodes():
    # 딕셔너리 1개씩 생성하여 넣기
    tmp = dict()
    tmp['id'] = num
    tmp['label'] = name
    tmp['degree'] = degree_list[num][1]

    
    # 색상은 파랑색으로 지정
    tmp['color'] = "#97C2FC"
    
#     # 차수가 3이상이면 빨간색 지정
#     if(degree_list[num][1] >= 3):
#         tmp['color'] = "#FB7E81"
#     else:
#         tmp['color'] = "#97C2FC"
        
    nodes_list.append(tmp)   # {id: 0, label: 'hemp'}형태
    
    reverse_graph[name] = num   # {'hemp':0, 'cannabi':1}형태, 목적: 'hemp'검색시 바로 노드 번호 확인(편의성 목적)
    num += 1

In [31]:
nodes_list

[{'id': 0, 'label': '우려', 'degree': 1, 'color': '#97C2FC'},
 {'id': 1, 'label': '시간', 'degree': 2, 'color': '#97C2FC'},
 {'id': 2, 'label': '커피', 'degree': 2, 'color': '#97C2FC'},
 {'id': 3, 'label': '여성', 'degree': 1, 'color': '#97C2FC'},
 {'id': 4, 'label': '홍차', 'degree': 4, 'color': '#97C2FC'},
 {'id': 5, 'label': '영국', 'degree': 4, 'color': '#97C2FC'},
 {'id': 6, 'label': '한국', 'degree': 2, 'color': '#97C2FC'},
 {'id': 7, 'label': '인도', 'degree': 2, 'color': '#97C2FC'},
 {'id': 8, 'label': '티백', 'degree': 3, 'color': '#97C2FC'},
 {'id': 9, 'label': '경우', 'degree': 3, 'color': '#97C2FC'},
 {'id': 10, 'label': '중국', 'degree': 1, 'color': '#97C2FC'},
 {'id': 11, 'label': '소비', 'degree': 1, 'color': '#97C2FC'},
 {'id': 12, 'label': '청나라', 'degree': 1, 'color': '#97C2FC'},
 {'id': 13, 'label': '국내', 'degree': 3, 'color': '#97C2FC'},
 {'id': 14, 'label': '녹차', 'degree': 1, 'color': '#97C2FC'},
 {'id': 15, 'label': '카페', 'degree': 3, 'color': '#97C2FC'},
 {'id': 16, 'label': '귀족', 'degre

In [32]:
graph_nodes += str(nodes_list) + ";"
print(graph_nodes)

var nodes = [{'id': 0, 'label': '우려', 'degree': 1, 'color': '#97C2FC'}, {'id': 1, 'label': '시간', 'degree': 2, 'color': '#97C2FC'}, {'id': 2, 'label': '커피', 'degree': 2, 'color': '#97C2FC'}, {'id': 3, 'label': '여성', 'degree': 1, 'color': '#97C2FC'}, {'id': 4, 'label': '홍차', 'degree': 4, 'color': '#97C2FC'}, {'id': 5, 'label': '영국', 'degree': 4, 'color': '#97C2FC'}, {'id': 6, 'label': '한국', 'degree': 2, 'color': '#97C2FC'}, {'id': 7, 'label': '인도', 'degree': 2, 'color': '#97C2FC'}, {'id': 8, 'label': '티백', 'degree': 3, 'color': '#97C2FC'}, {'id': 9, 'label': '경우', 'degree': 3, 'color': '#97C2FC'}, {'id': 10, 'label': '중국', 'degree': 1, 'color': '#97C2FC'}, {'id': 11, 'label': '소비', 'degree': 1, 'color': '#97C2FC'}, {'id': 12, 'label': '청나라', 'degree': 1, 'color': '#97C2FC'}, {'id': 13, 'label': '국내', 'degree': 3, 'color': '#97C2FC'}, {'id': 14, 'label': '녹차', 'degree': 1, 'color': '#97C2FC'}, {'id': 15, 'label': '카페', 'degree': 3, 'color': '#97C2FC'}, {'id': 16, 'label': '귀족', 'degree': 

In [33]:
print(reverse_graph) # graph_nodes를 뒤집어서 저장, 예를 들어  label명 : id명

{'우려': 0, '시간': 1, '커피': 2, '여성': 3, '홍차': 4, '영국': 5, '한국': 6, '인도': 7, '티백': 8, '경우': 9, '중국': 10, '소비': 11, '청나라': 12, '국내': 13, '녹차': 14, '카페': 15, '귀족': 16, '판매': 17, '설탕': 18, '사용': 19}


In [34]:
edges_list = []

edge_max = 0.0
edge_min = 7.0

for edge in G.edges(data='weight'):
#     print("step: " + edge[2])
    tmp = float(edge[2])

    #최댓값 저장
    if(edge_max <= tmp):
        edge_max = tmp
    
    #최솟값 저장
    if(edge_min >= tmp):
        edge_min = tmp
        
#     print("edge_max: " + str(edge_max))
#     print("edge_min: " + str(edge_min))
#     print()

In [35]:
# edges는 {'hemp', 'cannabi', '3.21'}형태
for edge in G.edges(data='weight'):
    tmp = dict()
    tmp['from']   = reverse_graph.get(edge[0])     # 'hemp'검색시 인덱스값
    tmp['to']     = reverse_graph.get(edge[1])     # 'cannabi'검색시 인덱스값
    tmp['label']  = edge[2]                        # 가중치 저장
    
    length = 7.0 - float(edge[2])
    convert_edge_min = 7.0 - edge_max
    minmax_scale = ((length - convert_edge_min) / (edge_max - edge_min)) * 500        # 길이를 min max 스케일링
    tmp['value']  = round(minmax_scale, 2)         # Edge 두께를 가중치의 크기에 따라 변경(숫자가 클수록 두껍게)
    
    length = float(edge[2])
    minmax_scale = ((length - edge_min)/(edge_max - edge_min)) * 500
    tmp['length'] = round(minmax_scale +220, 2)    # Edge 길이를 min max 스케일링
    
    tmp['color'] = "#d3d3d3"                      

    edges_list.append(tmp)

In [36]:
graph_edges += str(edges_list) + ";"
print(graph_edges)

var edges = [{'from': 0, 'to': 1, 'label': '3.79', 'value': 500.0, 'length': 220.0, 'color': '#d3d3d3'}, {'from': 1, 'to': 6, 'label': '4.69', 'value': 337.55, 'length': 382.45, 'color': '#d3d3d3'}, {'from': 2, 'to': 3, 'label': '3.79', 'value': 500.0, 'length': 220.0, 'color': '#d3d3d3'}, {'from': 2, 'to': 9, 'label': '5.78', 'value': 140.79, 'length': 579.21, 'color': '#d3d3d3'}, {'from': 4, 'to': 5, 'label': '4.68', 'value': 339.35, 'length': 380.65, 'color': '#d3d3d3'}, {'from': 4, 'to': 16, 'label': '5.47', 'value': 196.75, 'length': 523.25, 'color': '#d3d3d3'}, {'from': 4, 'to': 13, 'label': '5.56', 'value': 180.51, 'length': 539.49, 'color': '#d3d3d3'}, {'from': 4, 'to': 15, 'label': '5.68', 'value': 158.84, 'length': 561.16, 'color': '#d3d3d3'}, {'from': 5, 'to': 7, 'label': '4.78', 'value': 321.3, 'length': 398.7, 'color': '#d3d3d3'}, {'from': 5, 'to': 10, 'label': '5.02', 'value': 277.98, 'length': 442.02, 'color': '#d3d3d3'}, {'from': 5, 'to': 12, 'label': '5.08', 'value': 2

In [37]:
graph_data = graph_nodes + "\n\n" + graph_edges

In [38]:
print(graph_data)

var nodes = [{'id': 0, 'label': '우려', 'degree': 1, 'color': '#97C2FC'}, {'id': 1, 'label': '시간', 'degree': 2, 'color': '#97C2FC'}, {'id': 2, 'label': '커피', 'degree': 2, 'color': '#97C2FC'}, {'id': 3, 'label': '여성', 'degree': 1, 'color': '#97C2FC'}, {'id': 4, 'label': '홍차', 'degree': 4, 'color': '#97C2FC'}, {'id': 5, 'label': '영국', 'degree': 4, 'color': '#97C2FC'}, {'id': 6, 'label': '한국', 'degree': 2, 'color': '#97C2FC'}, {'id': 7, 'label': '인도', 'degree': 2, 'color': '#97C2FC'}, {'id': 8, 'label': '티백', 'degree': 3, 'color': '#97C2FC'}, {'id': 9, 'label': '경우', 'degree': 3, 'color': '#97C2FC'}, {'id': 10, 'label': '중국', 'degree': 1, 'color': '#97C2FC'}, {'id': 11, 'label': '소비', 'degree': 1, 'color': '#97C2FC'}, {'id': 12, 'label': '청나라', 'degree': 1, 'color': '#97C2FC'}, {'id': 13, 'label': '국내', 'degree': 3, 'color': '#97C2FC'}, {'id': 14, 'label': '녹차', 'degree': 1, 'color': '#97C2FC'}, {'id': 15, 'label': '카페', 'degree': 3, 'color': '#97C2FC'}, {'id': 16, 'label': '귀족', 'degree': 

In [39]:
f = open("./results/result_webgraph/data/" + file_name + ".js", 'w', encoding = 'utf8')
f.write(graph_data)
f.close()
print(file_name + ".js 파일 생성 완료")

홍차.js 파일 생성 완료


### 2. keyword.html

In [40]:
html_top = """
<!doctype html>
<html>
<head>
	<title>Knowledge Structure</title>
    """

In [41]:
vis_location = """<script type="text/javascript" src="../../lib/drawing/vis.js"></script>
    """

In [42]:
js_location = """<script src="../../results/result_webgraph/data/""" + file_name + """.js" charset="utf-8">
    </script>
    <style type="text/css">#mynetwork 
        {width: 100%;height: 1000px;border: 1px solid lightgray;}
    </style>
</head>
<body>
    """

In [43]:
title = """<p> Cheonsol of Knowledge Structure.</p>"""

In [44]:
body = """
    <div id="mynetwork"></div>
	<script type="text/javascript">
		var container = document.getElementById('mynetwork');
		var data = { nodes: nodes, edges: edges }; """

In [45]:
option = """
        var options = 
        { "nodes": {"shape" : "circle",
                    "font":{"size": 35}},
        "edges": {"scaling":{"max":70}, "font": {"size" : 40}, 
            "smooth": {"type":"continuous","roundness": 0.5}},
            "physics": {"barnesHut": {"centralGravity": 0.05},
            "minVelocity": 0.75}};
        """

In [46]:
bottom = """var network = new vis.Network(container, data, options);
	</script>
</body>
</html>
"""

In [47]:
html_text = html_top + vis_location + js_location + title + body + option + bottom

In [48]:
html_text

'\n<!doctype html>\n<html>\n<head>\n\t<title>Knowledge Structure</title>\n    <script type="text/javascript" src="../../lib/drawing/vis.js"></script>\n    <script src="../../results/result_webgraph/data/홍차.js" charset="utf-8">\n    </script>\n    <style type="text/css">#mynetwork \n        {width: 100%;height: 1000px;border: 1px solid lightgray;}\n    </style>\n</head>\n<body>\n    <p> Cheonsol of Knowledge Structure.</p>\n    <div id="mynetwork"></div>\n\t<script type="text/javascript">\n\t\tvar container = document.getElementById(\'mynetwork\');\n\t\tvar data = { nodes: nodes, edges: edges }; \n        var options = \n        { "nodes": {"shape" : "circle",\n                    "font":{"size": 35}},\n        "edges": {"scaling":{"max":70}, "font": {"size" : 40}, \n            "smooth": {"type":"continuous","roundness": 0.5}},\n            "physics": {"barnesHut": {"centralGravity": 0.05},\n            "minVelocity": 0.75}};\n        var network = new vis.Network(container, data, opti

In [49]:
html_file = open('./results/result_webgraph/' + file_name + '.html', 'w')
html_file.write(html_text)
html_file.close()
print(file_name + ".html 파일 생성 완료")

홍차.html 파일 생성 완료
